In [2]:
import numpy as np
import pandas as pd

In [3]:
crop_data = pd.read_csv("data.csv")

In [4]:
crop_data

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andhra Pradesh,ANANTAPUR,1997,Kharif,Arhar/Tur,21400,2600.0
1,Andhra Pradesh,ANANTAPUR,1997,Kharif,Bajra,1400,500.0
2,Andhra Pradesh,ANANTAPUR,1997,Kharif,Castor seed,1000,100.0
3,Andhra Pradesh,ANANTAPUR,1997,Kharif,Cotton(lint),7300,9400.0
4,Andhra Pradesh,ANANTAPUR,1997,Kharif,Dry chillies,3700,7100.0
...,...,...,...,...,...,...,...
9623,Andhra Pradesh,WEST GODAVARI,2014,Rabi,Tobacco,28046,89107.0
9624,Andhra Pradesh,WEST GODAVARI,2014,Rabi,Tomato,224,3035.0
9625,Andhra Pradesh,WEST GODAVARI,2014,Rabi,Urad,5418,4833.0
9626,Andhra Pradesh,WEST GODAVARI,2014,Whole Year,Coconut,21729,718991000.0


In [6]:
crop_data["District_Name"].unique()

array(['ANANTAPUR', 'CHITTOOR', 'EAST GODAVARI', 'GUNTUR', 'KADAPA',
       'KRISHNA', 'KURNOOL', 'PRAKASAM', 'SPSR NELLORE', 'SRIKAKULAM',
       'VISAKHAPATANAM', 'VIZIANAGARAM', 'WEST GODAVARI'], dtype=object)

In [ ]:
import requests
import pandas as pd
from datetime import datetime

district_coords = {
    'ANANTAPUR': {'lat': 14.68, 'lon': 77.61}, 
    'CHITTOOR': {'lat': 13.21, 'lon': 79.10}, 
    'EAST GODAVARI': {'lat': 17.01, 'lon': 81.78},
    'GUNTUR': {'lat': 16.30, 'lon': 80.46},
    'KADAPA': {'lat': 14.48, 'lon': 78.82},
    'KRISHNA': {'lat': 16.51, 'lon': 80.65},
    'KURNOOL': {'lat': 15.83, 'lon': 78.04},
    'PRAKASAM': {'lat': 15.50, 'lon': 80.04}, 
    'SPSR NELLORE': {'lat': 14.45, 'lon': 79.99}, 
    'SRIKAKULAM': {'lat': 18.30, 'lon': 83.90},
    'VISAKHAPATANAM': {'lat': 17.68, 'lon': 83.20}, 
    'VIZIANAGARAM': {'lat': 18.12, 'lon': 83.41}, 
    'WEST GODAVARI': {'lat': 16.54, 'lon': 81.52}
}
START_YEAR = 1997
END_YEAR = 2014
start_date_str = f"{START_YEAR}-01-01"
end_date_str = f"{END_YEAR}-12-31"
WEATHER_VARIABLES = "precipitation_sum,temperature_2m_max,temperature_2m_min"
all_weather_data = []
print("Starting to fetch weather data for all districts...")
for district, coords in district_coords.items():
    print(f"Fetching data for {district}...")
    api_url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={coords['lat']}&longitude={coords['lon']}&"
        f"start_date={start_date_str}&end_date={end_date_str}&"
        f"daily={WEATHER_VARIABLES}&"
        f"timezone=Asia/Kolkata"
    )
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        daily_df = pd.DataFrame(data['daily'])
        
        daily_df['District'] = district.upper()
        
        daily_df.rename(columns={
            'time': 'Date',
            'precipitation_sum': 'Rainfall_mm',
            'temperature_2m_max': 'Temp_Max_C',
            'temperature_2m_min': 'Temp_Min_C'
        }, inplace=True)
        
        all_weather_data.append(daily_df)
        
    except requests.exceptions.RequestException as e:
        print(f"  > Error fetching data for {district}: {e}")

if all_weather_data:
    final_weather_df = pd.concat(all_weather_data, ignore_index=True)
    
    output_filename = 'weather.csv'
    final_weather_df.to_csv(output_filename, index=False)
    
    print(f"\nAll weather data successfully fetched and saved to '{output_filename}'")
else:
    print("\nNo weather data was fetched. Please check your configuration and network connection.")

Starting to fetch weather data for all districts...
Fetching data for ANANTAPUR...
Fetching data for CHITTOOR...
Fetching data for EAST GODAVARI...
Fetching data for GUNTUR...
Fetching data for KADAPA...
Fetching data for KRISHNA...
Fetching data for KURNOOL...
Fetching data for PRAKASAM...
Fetching data for SPSR NELLORE...
Fetching data for SRIKAKULAM...
Fetching data for VISAKHAPATANAM...
Fetching data for VIZIANAGARAM...
Fetching data for WEST GODAVARI...

All weather data successfully fetched and saved to 'weather.csv'
